<a href="https://colab.research.google.com/github/Lehula1/divergGit/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install plotly
! pip install numpy
! pip install yfinance
! pip install pandas_ta
! pip install matplotlib
! pip install ta
! pip install finta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=4ed148241edb512373d4680cd1762039ab386c0019654c043e3e8f8bb8e1c054
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pub

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly
from pandas_datareader import data as pdr
import pandas_ta as ta
import yfinance as yf
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from collections import deque
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import datetime as dt



"""
data1 = yf.download(tickers='SPY', period='30d', interval='1d')

fig = go.Figure(data=[go.Candlestick(x=data1.index,
                                     open=data1['Open'],
                                     high=data1['High'],
                                     low=data1['Low'],
                                     close=data1['Close'])])


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()
"""


"\ndata1 = yf.download(tickers='SPY', period='30d', interval='1d')\n\nfig = go.Figure(data=[go.Candlestick(x=data1.index,\n                                     open=data1['Open'],\n                                     high=data1['High'],\n                                     low=data1['Low'],\n                                     close=data1['Close'])])\n\n\nfig.update_layout(xaxis_rangeslider_visible=False)\nfig.show()\n"

In [29]:
ticker = 'ACLS'
yfObj = yf.Ticker(ticker)
data = yfObj.history(period='3mo', interval='1d')

close = data['High'].values
dates = data.index

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

max_idx = argrelextrema(data['High'].values, np.greater, order=2)[0]
min_idx = argrelextrema(data['Low'].values, np.less, order=2)[0]

PriceH = 0
PriceHH = 0
# Get K consecutive higher peaks
K = 2
highs = data.iloc[max_idx]['High']

extrema = []
count = 0
ex_deque = deque(maxlen=K)
for i, idx in enumerate(max_idx):
  if i == 0:
    ex_deque.append(idx)
    continue
  if highs[i] < highs[i-1]:
    ex_deque.clear()
   
  ex_deque.append(idx)

  if len(ex_deque) == K:
    # K-consecutive higher highs found
    extrema.append(ex_deque.copy())
    #extrema count++++++++++++++++++++++++++++++++++++++++++++++++
    #print(extrema[count][0])
    #print(extrema[count][1])
    count += 1

test = data.iloc[extrema[0][0]]['High']
"""
#print(highs)
print(data.iloc[max_idx])
print(ex_deque)
print(test)
print(extrema.index)
print(extrema[0][1])
print(extrema[1])
print(extrema[-1][1])
print(extrema)
print(max_idx)
print(data.index[max_idx])
print(data.iloc[max_idx,1])
"""

data.ta.stoch(high='high', low='low', k=14, d=3, append=True)

# Avoid case-sensitive issues for accessing data.
# Optional if using pandas_ta


data.columns = [x.lower() for x in data.columns]

max_stoch = argrelextrema(data['stochk_14_3_3'].values, np.greater, order=2)[0]


stoch_highs = data.iloc[max_stoch]['stochk_14_3_3']


extremaStoch = []
count1 = 0
stoch_deque = deque(maxlen=K)
for j, sto in enumerate(max_stoch):
  if j == 0:
    stoch_deque.append(sto)
    continue
  if stoch_highs[j] > stoch_highs[j-1] or data['high'].values[max_stoch[j-1]] > data['high'].values[max_stoch[j]]:

      stoch_deque.clear()
      print(data['high'].values[max_stoch[j-1]])
      print(data['high'].values[max_stoch[j]]) 


  stoch_deque.append(sto)
  
  if len(stoch_deque) == K:
    # K-consecutive higher highs found
    extremaStoch.append(stoch_deque.copy())
  
    
    #print(extremaStoch[count1][0])
    
    #print(extremaStoch[count1][1])
    count1 += 1







""" 
candlestickHigh = data['high'].values[max_stoch[1]]
test = argrelextrema(data['high'].values, np.greater, order=2)[0]
print(candlestickHigh)
print(close)
print(extremaStoch)
print(stoch_highs)
print(data.iloc[extremaStoch[1]])

print(data['high'].values)
print(highs)
print(max_idx)

print(data['stochk_14_3_3'].values) #----------------------------------------------------------------------
print(data.iloc[max_stoch]['stochk_14_3_3'])

"""

# Create our primary chart
# the rows/cols arguments tell plotly we want two figures
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,row_heights=[1,.5], vertical_spacing = 0.01)  

# Create our Candlestick chart with an overlaid price line
fig.append_trace(
    go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        showlegend=False
    ), row=1, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_idx],
        y=data.iloc[max_idx,1]+5.5,
        marker=dict(
            symbol='cross',
            color='green',
            size=10)))

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[min_idx],
        y=data.iloc[min_idx,2]-5.5,
        marker=dict(
            symbol='cross',
            color='red',
            size=10)))

fig.add_trace(
    go.Scatter(
        x=[data.index[extrema[-1][0]], data.index[extrema[-1][1]]],
        y=[data.iloc[extrema[-1][0],1], data.iloc[extrema[-1][1],1]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')))

# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=data.index,
        y=data['stochk_14_3_3'],
        line=dict(color='#ff9900', width=2),
        name='fast'), row=2, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_stoch],
        y=data.iloc[max_stoch]['stochk_14_3_3'],
        marker=dict(
            symbol='cross',
            color='red',
            size=10)), row=2, col=1)

fig.add_trace(
    go.Scatter(
        x=[data.index[extremaStoch[-1][0]], data.index[extremaStoch[-1][1]]],
        y=[data.iloc[extremaStoch[-1][0],7], data.iloc[extremaStoch[-1][1],7]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')),row=2, col=1)




# Extend our y-axis a bit
fig.update_yaxes(range=[-10, 110], row=2, col=1)

# Add upper/lower bounds
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)

# Add overbought/oversold
fig.add_hline(y=20, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=80, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"])] #hide weekends  
)

fig.update_layout(xaxis_rangeslider_visible=False,  width=1500,
     margin=dict(
        l=50,
        r=50,
        b=10,
        t=10,
        pad=4))



# View our chart in the system default HTML viewer (Chrome, Firefox, etc.)


136.32000732421875
135.25
135.25
131.60000610351562
131.60000610351562
127.51000213623047


In [4]:
# Add some indicators


fig=go.Figure()

fig = plotly.subplots.make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01, 
                    row_heights=[0.5,0.4])

fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close']))

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_idx],
        y=data.iloc[max_idx,1]+2,
        marker=dict(
            symbol='cross',
            color='green',
            size=10,
         
        )
    )
)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[min_idx],
        y=data.iloc[min_idx,2]-2,
        marker=dict(
            symbol='cross',
            color='red',
            size=10)))

fig.add_trace(
    go.Scatter(
        x=[data.index[extrema[-1][0]], data.index[extrema[-1][1]]],
        y=[data.iloc[extrema[-1][0],1], data.iloc[extrema[-1][1],1]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')))




fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"])] #hide weekends  
)

fig.update_layout(xaxis_rangeslider_visible=False)



KeyError: ignored

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(data['High'])
_ = [plt.plot(dates[i], close[i], c=colors[1]) for i in extrema]


#create figure
plt.figure(figsize=(20,5))

plt.scatter(data.iloc[max_idx].index, data.iloc[max_idx]['High']+.5,
  label='Maxima', s=100, color='green', marker='+')
plt.scatter(data.iloc[min_idx].index, data.iloc[min_idx]['Low']-.5,
  label='Minima', s=100, color='red', marker='+')

#define width of candlestick elements
width = .6
width2 = .1

#define up and down prices
up = data[data['Close']>=data['Open']]
down = data[data['Close']<data['Open']]

#define colors to use
col1 = 'green'
col2 = 'red'

#plot up prices
plt.bar(up.index,up['Close']-up['Open'],width,bottom=up['Open'],color=col1)
plt.bar(up.index,up['High']-up['Close'],width2,bottom=up['Close'],color=col1)
plt.bar(up.index,up['Low']-up['Open'],width2,bottom=up['Open'],color=col1)

#plot down prices
plt.bar(down.index,down['Close']-down['Open'],width,bottom=down['Open'],color=col2)
plt.bar(down.index,down['High']-down['Open'],width2,bottom=down['Open'],color=col2)
plt.bar(down.index,down['Low']-down['Close'],width2,bottom=down['Close'],color=col2)

#rotate x-axis tick labels
plt.xticks(rotation=45, ha='right')

#lines
_ = [plt.plot(dates[i], close[i], c=colors[1]) for i in extrema]

plt.legend()
#display candlestick chart
plt.show()